In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def load_data(params_path, angles_path, energies_path, names_params, drop_nan = False):
    params = np.load(params_path)
    angles = np.load(angles_path, allow_pickle = True)
    energies = np.load(energies_path, allow_pickle = True)
    
    data = {}
    for k in range(params.shape[0]):
        data[names[k]] = params[:,k]
    
    data['Tf'] = angles[:,0]
    data['Pf'] = angles[:,1]
    data['Ef/Ei'] = energies
    
    if(drop_nan):
        df = pd.DataFrame(data)
        return df.dropna(subset = ["Tf"], inplace=True) # for example 'Tf'
    return pd.DataFrame(data)

## Loading the dataframe with the data

In [ ]:
params_path = ''
angles_path = ''
energies_path = ''
names_params = ['Ei','Ti','Pi']

In [ ]:
df = load_data(params_path, angles_path, energies_path, names_params, drop_nan = False)

## Simple plottingfig

In [ ]:
fig, ax = plt.subplots(2,2)
ax[0,0].plot(df['Ti'],df['Ef/Ei'],'.')
ax[0,1].plot(df['Ti'],df['Tf'],'.')
ax[1,0].plot(df['Pi'],df['Ef/Ei'],'.')
ax[1,1].plot(df['Ei'],df['Ef/Ei'],'.')

## Sensibility analysis
Using [SALib](https://salib.readthedocs.io/en/latest/index.html).

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
from .modules.plotting import plot_sobol

In [ ]:
problem = {
    'num_vars': 2,
    'names': ['Ei', 'Ti'],
    'bounds': [[1.0, 150], # eV
                [-90, 90]]
}

In [ ]:
param_values = df[names_params].values

In [ ]:
Y = np.zeros((param_values.shape[0],2))

In [ ]:
Y[:,0] = df['Ef/Ei']
Y[:,1] = df['Tf']

In [ ]:
Si_energy = sobol.analyze(problem, Y[:,0])
Si_polar_angle = sobol.analyze(problem, Y[:,1])

In [ ]:
plot_sobol(Si_energy['S1'],Si_energy['ST'], title = 'Sobol Analysis - Out energy')

In [ ]:
plot_sobol(Si_polar_angle['S1'],Si_polar_angle['ST'], title = 'Sobol Analysis - Out polar angle')

# Machine Learning - trial 1

In [ ]:
# importation des données 
import torch
import numpy as np
import tqdm
from torch import optim
from torch import nn
from torchvision import datasets, transforms

In [ ]:
# splitting 
def preprocessed(data, start_idx_ground_truth, splitting = 0.6):
    s = data.shape
    np.random.shuffle(data)
    splittinglimit = int(splitting*s[0])
    
    training_set = data[:splittinglimit]
    testing_set = data[splittinglimit:]
        
        # converting to torch tensors
    x_train = torch.Tensor(training_set[:,:start_idx_ground_truth])
    y_train = torch.Tensor(training_set[:,start_idx_ground_truth:])
    x_test = torch.Tensor(testing_set[:,:start_idx_ground_truth])
    y_test = torch.Tensor(testing_set[:,start_idx_ground_truth:])
    
        # normalizing
    # x_test = nn.functional.normalize(x_test, p=2, dim=0)
    # x_train = nn.functional.normalize(x_train, p=2, dim=0)
    
        # creating datasetabs    
    training_dataset = torch.utils.data.TensorDataset(x_train,y_train)
    testing_dataset = torch.utils.data.TensorDataset(x_test,y_test)
    
    return (training_dataset, testing_dataset)

In [ ]:
training_dataset, testing_dataset = preprocessed(df[['Ei','Ti','Ef/Ei','Tf']].values, start_idx_ground_truth = 2, splitting = 0.9)

In [ ]:
# definition of a testloader
#testloader = torch.utils.data.DataLoader(test_data, len(test_data), shuffle=True)

def train_epoch(net, opt, criterion, trainloader, batch_size=50): # pour entraîner sur un epoch
    net.train()
    losses = []
    for x_batch, y_batch in trainloader:
        opt.zero_grad()
        # Forward
        y_comp = net(x_batch)
        # Compute diff
        loss = criterion(y_comp, y_batch)
        # Compute gradients
        loss.sum().backward()
        # update weights
        opt.step()
        losses.append(loss.data.numpy())
    return losses

def accuracy(net, dataset): # pour calculer la précision
    net.eval() # pass the model to evaluation mode
    correct = 0
    total = 0
    sqe = 0
    count = 0
    for data in dataset:
        points, ground_truth = data
        outputs = net(points)
        #print(outputs, labels)
        if(np.isnan(ground_truth.detach().numpy()).any()):
            count += 1
        else:
            sum_err = (outputs-ground_truth)**2 # problem : we have nan at this point
            sqe += np.mean(sum_err.detach().numpy())
    net.train() # pass the model to training mode
    #print(count)
    return sqe

In [ ]:
class Net(nn.Module):
    def __init__(self, deepness = 10, weights_nb = 256):
        super(Net, self).__init__()
        self.inputlayer = nn.Linear(2,weights_nb)
        liste = []
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        for k in range(deepness):
            liste.append(nn.Linear(weights_nb,weights_nb))
            liste.append(self.relu)
        self.hiddenlayers = nn.Sequential(*liste)
        self.outputlayer = nn.Linear(weights_nb,2)
        
    def forward(self, x):
        # intput layer
        x = self.inputlayer(x)
        x = self.relu(x)
        # hidden layers
        x = self.hiddenlayers(x)
        # output layer
        x = self.outputlayer(x)
       #x = self.sigmoid(x)

        return x

In [ ]:
def test_model(train_data, test_data, model, batch_size = 64, epochs = 10, lr = 1e-4):
    """
    this function returns a list of precision - 1 precision / epoch - for the given training and testing data, for the given model
    """
    testloader = torch.utils.data.DataLoader(test_data, 1, shuffle=False)
    trainloader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
    net = model
    
    # criterion for the last and optimizer
    def criterion(x,y):
        n = len(x)
        s = 0
        for k in range(n):
            s+=(x[k]-y[k])*(x[k]-y[k]) # mean square error
        return s/n
    
    opt = torch.optim.Adam(net.parameters(),lr,betas=(0.9, 0.999),eps=1e-08)
    accuracy_list = []
    #accuracy_list_train = []
    for k in tqdm.tqdm(range(epochs)):
        net.train()
        train_epoch(net, opt, criterion, trainloader, batch_size = batch_size)
        prec=accuracy(net, dataset = testloader)
        if(k%10==0):
            print(f'{k} : accuracy = {prec}')
        accuracy_list.append(prec)
        #accuracy_list_train.append(accuracy(net, dataset = trainloader))
    return (net , accuracy_list) # this way we can keep the trained net and test it some more after

In [ ]:
net = Net(deepness = 2, weights_nb = 16)
net, accuracy_list = test_model(training_dataset, testing_dataset, net, batch_size = 1, epochs = 100, lr=1e-2) # 1e-2 ~ 1% of the total value

In [ ]:
testloader = torch.utils.data.DataLoader(doe, 1, shuffle=False)
print(accuracy(net, testing_dataset))

In [ ]:
doe_valid = np.load('doe_valid.npy')
angles_valid = np.load('angles_valid.npy', allow_pickle = True)
energies_valid = np.load('ratio_energies_valid.npy', allow_pickle = True)